In [9]:
from __future__ import annotations

from pathlib import Path

import dill
import numpy as np
import rich
from matbench_discovery.enums import MbdKey
from matbench_discovery.metrics import stable_metrics
from pymatviz import Key
from tqdm.auto import tqdm

results_dir = Path(
    "/net/csefiles/coc-fung-cluster/nima/shared/repositories/jmp-backbone/notebooks/results/epoch0-step28722"
)
results = list(results_dir.glob("*.dill"))

predicted_eform_list: list[float] = []
true_eform_list: list[float] = []

predicted_ehull_list: list[float] = []
true_ehull_list: list[float] = []

for result_path in tqdm(results):
    with result_path.open("rb") as f:
        result = dill.load(f)

    # Ignore non-unique prototypes
    if not result["metadata"][Key.uniq_proto.value]:
        continue

    eform_pred = result["energy"]
    predicted_eform_list.append(eform_pred)

    eform_true = result["metadata"][MbdKey.e_form_wbm.value]
    true_eform_list.append(eform_true)

    hull_true = result["metadata"][MbdKey.each_true.value]
    true_ehull_list.append(hull_true)

    hull_pred = hull_true + (eform_pred - eform_true)
    predicted_ehull_list.append(hull_pred)

predicted_eform = np.array(predicted_eform_list)
true_eform = np.array(true_eform_list)

predicted_ehull = np.array(predicted_ehull_list)
true_ehull = np.array(true_ehull_list)

rich.print(stable_metrics(true_ehull, predicted_ehull))

  0%|          | 0/100 [00:00<?, ?it/s]

{
    'F1': 0.5,
    'DAF': 4.292929292929292,
    'Precision': 0.5555555555555556,
    'Recall': 0.45454545454545453,
    'Accuracy': 0.8823529411764706,
    'TPR': 0.45454545454545453,
    'FPR': 0.05405405405405406,
    'TNR': 0.9459459459459459,
    'FNR': 0.5454545454545454,
    'TP': 5,
    'FP': 4,
    'TN': 70,
    'FN': 6,
    'MAE': 0.1614175737040884,
    'RMSE': 0.26664921886092974,
    'R2': -0.6451545642797809
}